In [1]:
import torch
import math
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [12]:
import torch.nn as nn

class TinyVGG_Medical(nn.Module):
    def __init__(self, device):
        super().__init__()
        
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1,device=device),  # 224x224 → 224x224
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),  # 224 → 112

            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1,device=device),  # 112x112
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),  # 112 → 56

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1,device=device),  # 56x56
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),  # 56 → 28

            # Block 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1,device=device),  # 28x28
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),  # 28 → 14
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [13]:
model=TinyVGG_Medical(device="cuda")

In [14]:
x=torch.randn(1,3,224,224).to("cuda")
out=model(x)
out.shape

torch.Size([1, 4])

In [15]:
!nvidia-smi

Wed Jul 23 21:24:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2050        Off |   00000000:01:00.0 Off |                  N/A |
| N/A   40C    P0              3W /   50W |    2552MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
transformation=transforms.Compose([transforms.Resize(size=(244,244)),transforms.ToTensor()])
transformation_2=transforms.Compose([transforms.Resize(size=(244,244)),transforms.ToTensor()])

In [17]:
train_data=datasets.ImageFolder(root='/home/deepesh/intern_tumour/brain_tumour/Tumour/train',transform=transformation)
val_data=datasets.ImageFolder(root='/home/deepesh/intern_tumour/brain_tumour/Tumour/valid',transform=transformation_2)
test_data=datasets.ImageFolder(root='/home/deepesh/intern_tumour/brain_tumour/Tumour/test',transform=transformation_2)

In [18]:
train_loader=DataLoader(dataset=train_data,batch_size=32,shuffle=True)
val_loader=DataLoader(dataset=val_data,batch_size=32,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=32,shuffle=True)

In [19]:
optimizer=Adam(params=model.parameters(),lr=1e-4,weight_decay=1e-3)
lossfn=nn.CrossEntropyLoss()
scheduler=ReduceLROnPlateau(optimizer,mode='min',patience=3,factor=0.5)

In [20]:
def epoch_train():
    running_loss=0
    torch.cuda.empty_cache()
    model.train()
    for img,classes in (train_loader):
        img=img.to("cuda")
        classes=classes.to("cuda")
        out=model(img)
        optimizer.zero_grad()
        loss=lossfn(out,classes)
        loss.backward()
        optimizer.step()
        running_loss+=loss
        break
    return running_loss/len(train_loader)
    

In [21]:
!nvidia-smi

Wed Jul 23 21:24:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2050        Off |   00000000:01:00.0 Off |                  N/A |
| N/A   40C    P0              3W /   50W |    2552MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [22]:
epochs=30
loss=0
val_loss=1.0
train_losses=[]
val_losses=[]
for epoch in range(epochs):
    loss=epoch_train()
    model.eval()
    temp_loss=0
    with torch.no_grad():
        for img,lab in (val_loader):
            img=img.to("cuda")
            lab=lab.to("cuda")
            out=model(img)
            temp=lossfn(out,lab)
            temp_loss=temp+temp_loss
        temp_loss=temp_loss/len(val_loader)
        val_losses.append(temp_loss.detach().cpu().clone())
        train_losses.append(loss.detach().cpu().clone())
        scheduler.step(temp_loss)
        if(temp_loss<val_loss):
            val_loss=temp_loss
            torch.save(model.state_dict(),f"best_model-{val_loss:.2f}.pth")
            print("best model saved")
        
    clear_output(wait=True)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.title('Training and Validation Loss')
    plt.show()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x57600 and 50176x512)

In [285]:

model_name="/home/deepesh/intern_tumour/best_model-1.36.pth"
def test(model_name):
    test_model=custom_vgg(device="cuda")
    test_model.load_state_dict(torch.load(model_name))
    test_model.eval()
    correct=0
    total=0
    for img,lab in test_loader:
        img=img.to("cuda")
        lab=lab.to("cuda")
        out=model(img)
        pred=torch.argmax(out.data,dim=1)
        total+=lab.size(0)
        correct+=(pred==lab).sum().item()
    accuracy=100*correct/total
    print(f"Accuracy:{accuracy}")
        

In [286]:
test(model_name)

Accuracy:32.520325203252035
